In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

c:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = 'gpt2-xl'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [5]:
def get_response_from_corpus(user_input, corpus_df, tokenizer):
   """Buscar la respuesta en el corpus basada en la pregunta del usuario."""
   user_tokens = tokenizer.encode(user_input.lower(), add_special_tokens=False)

   for index, row in corpus_df.iterrows():
      pregunta = row['Pregunta'].lower()
      respuesta = row['Respuesta']

   for token in user_tokens:
      if token in tokenizer.encode(pregunta, add_special_tokens=False):
         return respuesta

   return None

In [11]:
def setup_chatbot():
   # Load the pre-trained GPT-2 model and tokenizer

   # Load your corpus from the CSV file
   corpus_df = pd.read_csv('../data/corpus_turims.csv', encoding='utf-8')

   print("Chatbot: ¡Hola! Soy un chatbot. ¿En qué puedo ayudarte hoy?")

   while True:
      user_input = input("Tú: tienes alguna consulta el día de hoy?")
      if user_input.lower() == 'exit':
         print("Chatbot: Hasta luego. ¡Que tengas un buen día!")
         break

      # Buscar la respuesta en el corpus basada en la pregunta del usuario
      response = get_response_from_corpus(user_input, corpus_df, tokenizer)

      if response:
         print("Chatbot:", response)
      else:
         print("Chatbot: Lo siento, no tengo información sobre eso en mi corpus.")

In [12]:
setup_chatbot()

Chatbot: ¡Hola! Soy un chatbot. ¿En qué puedo ayudarte hoy?


NameError: name 'tokenizer' is not defined